In [1]:
%load_ext autoreload
%autoreload 2
#from lib.multitasking_lib import *

In [7]:
######################
# Initiate Backtest  #
######################
from lib.multitasking_lib import *
cache_type ='backtest_uni'
my_cache = cache_state('backtest_uni')
#my_cache.flushall()
def backtest_unitest(stocks):
    stocks = stocks
    sl = 1
    target = 1
    qty = 10
    algo = ""
    fromDate = '2019-4-8'
    toDate = '2019-4-11'
    freq = '1T'
    algo_name = 'haikin_1'
    backtest_msg={'stock':stocks,'sl':sl,'target':target,'qty':qty,'algo':algo_name,'fromDate':fromDate,'toDate':toDate,'freq':freq}

    try:
        store_algo(algo, algo_name)
    except:
        perror('Something went wrong while saving algo')

    #pdebug1(backtest_msg)
    # Step 2: Store the stock name under backtest in the redis cache
    #for stock in stocks:
    #    redis_conn.set('stock',stock) #TODO: replace

    # Step 4: Done is set to 0: Backtest is in progress, will be resetted by backtest job
    redis_conn.set('done'+cache_type,0)
    # Step 5: Send the msg to backtest thread to initiate the back test
    #pdebug(json.dumps(backtest_msg))
    redis_conn.publish('kite_simulator'+cache_type,json.dumps(backtest_msg))
    
##########################
# Unit Test              #
##########################

open('log/freedom.log','r+')
#log_file.close()

logger.setLevel(10)
loggerT.setLevel(21)
redis_conn.set('done'+cache_type,1)

try:
    if backtest_manager.job.is_alive():
        backtest_manager.job.terminate()
        time.sleep(0.1)
except:
    pass
finally:
    backtest_manager = threadManager(cache_type, ["kite_simulator","ohlc_tick_handler"], [kite_simulator, ohlc_tick_handler])
    
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
print(jobs)


DEBUG:	Cache initialized for: backtest_uni
DEBUG:	Cache initialized for: backtest_uni


True
['kite_simulatorbacktest_web', 'ohlc_tick_handlerlive', 'freedom_initfreedom', 'ohlc_tick_handlerbacktest_web', 'order_handlerlive', 'tick_resamplerlive']
[<Process(Process-2, started)>]


INFO:	Starting Handler: kite_simulator
INFO:	Starting Handler: ohlc_tick_handler
DEBUG:	kite_simulator: 1
ERROR:	kite_simulator: Invalid msg: 1
DEBUG:	kite_simulator: {"stock": ["TCS"], "sl": 1, "target": 1, "qty": 10, "algo": "haikin_1", "fromDate": "2019-4-8", "toDate": "2019-4-11", "freq": "1T"}
DEBUG:	Trade_init: TCS
INFO:	TCSbacktest_uni=>{'stock': 'TCS', 'qty': '0', 'SL %': '0.0', 'TP %': '0.0', 'amount': '0', 'price': '0.0', 'P&L': '0.0', 'P&L %': '0.0', 'Total P&L': '0.0', 'Total P&L %': '0.0', 'low': '0.0', 'sl': '0.0', 'ltp': '0.0', 'ltp %': '0.0', 'tp': '0.0', 'high': '0.0', 'last_processed': '1999-01-01', 'state': 'INIT', 'mode': 'PAUSE', 'algo': '', 'freq': '1D', 'hdf_freq': 'day'}
TRADE: B: EN:  : 2019-04-08 09:34 : TCS -> 2049.1
TRADE: S: EX:  : 2019-04-08 10:13 : TCS -> 2045.7
TRADE: B: EN:  : 2019-04-08 10:33 : TCS -> 2042.55
TRADE: S: EX:  : 2019-04-08 10:36 : TCS -> 2044.15
TRADE: S: EN:  : 2019-04-08 10:39 : TCS -> 2047.0
TRADE: B: EX:  : 2019-04-08 10:54 : TCS -> 2

In [8]:
backtest_unitest(['TCS'])

ERROR:	Something went wrong while saving algo


In [6]:
my_cache.delete('msgBufferQueue'+cache_postfix)
my_cache.set('TCSbacktest_uniTICK',pd.DataFrame().to_json())

True

In [5]:
if backtest_manager.job.is_alive():
    backtest_manager.job.terminate()
backtest_manager.job.is_alive()
redis_conn.pubsub_channels()

['kite_simulatorbacktest_web',
 'ohlc_tick_handlerlive',
 'freedom_initfreedom',
 'ohlc_tick_handlerbacktest_web',
 'order_handlerlive',
 'tick_resamplerlive']

In [28]:
backtest_unitest(['TCS'])

ERROR:	Something went wrong while saving algo


In [ ]:
backtest_unitest(['WIPRO','TCS'])

In [2]:
from lib.charting_lib import *
cache_type = 'backtest_uni'
fig = freedom_chart('TCS', cache_type)
fig

NameError: name 'pattern' is not defined

In [3]:
dict(bounds=["sat", "mon"])

{'bounds': ['sat', 'mon']}

In [58]:
cache_state('backtest_uni').getValue()

DEBUG:	Cache initialized for: backtest_uni


,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq
0,WIPRO,10,1,1,0,0,0,0.0,0,0.0,...,293.65,0.0,0.0,299.45,2019-05-02 00:00,SCANNING,PAUSE,haikin_1,1D,day
1,TCS,10,1,1,0,0,0,0.0,0,0.0,...,2215.4,0.0,0.0,2266.95,2019-05-02 00:00,SCANNING,PAUSE,haikin_1,1D,day


In [ ]:
# Terminate all the processes
backtest_manager.job.terminate()
time.sleep(0.1)
print(backtest_manager.job.is_alive())
print(redis_conn.pubsub_channels())
jobs
jobs = []

In [ ]:
temp_file = pd.HDFStore("data/kite_cache.h5", mode="r")
temp_file.keys()

In [ ]:
# Long Strategies
def long_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
    #temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
    #temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
    #temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
    temp_df = temp_df | (REF(s.rsi,1) <=40)
    #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
    #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)

    return pd.DataFrame(  temp_df, columns=["buy"] )

# Short Strategies
def short_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
    #temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
    #temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
    #temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
    temp_df = temp_df | (REF(s.rsi,1) >=60)
    #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
    return pd.DataFrame( temp_df, columns=["sell"])



def long_breakout(s):
    temp_df = s.haCLOSE >= s.BBT.shift(1)
    temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
    return pd.DataFrame( temp_df , columns=["buy"])

def short_breakout(s):
    temp_df = s.haOPEN <= s.BBB.shift(1)
    temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
    return pd.DataFrame( temp_df , columns=["sell"])

def long_ha(s):
    temp_df = (REF(s.haCLOSE,3) < REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN)  
    #temp_df = temp_df & (s.rsi < 40 )
    return pd.DataFrame( temp_df , columns=["buy"])

def short_ha(s):
    temp_df = (REF(s.haCLOSE,3) > REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
    #temp_df = temp_df & (s.rsi > 60 )
    return pd.DataFrame( temp_df , columns=["sell"])
    


## Redis Console

In [ ]:
from redis import Redis
import json

conn = Redis(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)

In [ ]:
conn.xadd('msgBufferQueue',{'msg': 'next'})

In [ ]:
pd.set_option('display.max_rows', 50)
pd.read_json(conn.hget(stock+'_state','ohlc')).tail(50)

## Scratchpad

In [ ]:
redis_conn.pubsub_channels()

In [9]:
my_cache = cache_state('backtest_uni')

In [ ]:
df = my_cache.getValue()

In [13]:
pd.set_option('display.max_rows', 1000)
my_cache.getOHLC('TCS')

,close,high,low,open
2019-04-18 13:50:00,2144.00,2145.25,2144.00,2145.25
2019-04-18 13:51:00,2144.90,2145.00,2144.00,2144.00
2019-04-18 13:52:00,2143.75,2145.00,2143.75,2144.50
2019-04-18 13:53:00,2144.00,2144.00,2143.10,2143.75
2019-04-18 13:54:00,2144.70,2144.80,2143.80,2144.00
2019-04-18 13:55:00,2144.45,2144.80,2144.15,2144.70
2019-04-18 13:56:00,2144.80,2145.00,2144.45,2144.45
2019-04-18 13:57:00,2145.20,2145.50,2144.80,2144.80
2019-04-18 13:58:00,2145.95,2145.95,2145.05,2145.30
2019-04-18 13:59:00,2145.95,2145.95,2145.40,2145.40


In [ ]:
trades_df = my_cache.getTrades('TCS')
trades_df

In [ ]:
(total_profit, max_loss, max_profit, total_win, total_loss, max_winning_streak, max_loosing_streak, trade_log_df) = trade_analysis_raw(trades_df)

In [1]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [8]:
# GetData

logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-4-22'
toDate  = '2019-4-24'
freq = 'minute'
mydate = fromDate
exchange = 'NSE'
ohlc_data = {}
ohlc_data[stock] = getData(stock, fromDate, toDate, exchange, freq, False, stock)

In [6]:
ohlc_data[stock].head()

,close,high,low,open,volume
date,,,,,
2019-04-22 09:15:00,285.90,286.50,285.15,285.50,167257
2019-04-22 09:16:00,286.45,286.55,285.60,285.85,128478
2019-04-22 09:17:00,286.40,286.55,286.00,286.25,105224
2019-04-22 09:18:00,286.80,287.00,286.35,286.40,96006
2019-04-22 09:19:00,287.65,287.80,286.80,286.80,100153


In [14]:
from lib.data_model_lib import *
live_cache = cache_state('backtest_web')
live_cache.getValue()
#live_cache.getOHLC('TCS', '1D').head()

DEBUG:	Cache initialized for: backtest_web


,stock,qty,SL %,TP %,amount,price,P&L,P&L %,Total P&L,Total P&L %,...,ltp,ltp %,tp,high,last_processed,state,mode,algo,freq,hdf_freq
0,WIPRO,10,1,1,2855.0,0,35.0,12.110726643598616,8.000000000000114,0.28021015761821766,...,298.55,0.0,0,299.45,2019-04-30 00:00,SCANNING,PAUSE,default,1D,day
1,TCS,10,1,1,22030.0,0,-450.0,-20.852641334569043,-1003.4999999999991,-4.555152065365407,...,2260.35,0.0,0,2266.95,2019-04-30 00:00,SCANNING,PAUSE,default,1D,day


In [18]:
help(go.Ohlc)

Help on class Ohlc in module plotly.graph_objs:

class Ohlc(plotly.basedatatypes.BaseTraceType)
 |  Ohlc(arg=None, close=None, closesrc=None, customdata=None, customdatasrc=None, decreasing=None, high=None, highsrc=None, hoverinfo=None, hoverinfosrc=None, hoverlabel=None, hovertext=None, hovertextsrc=None, ids=None, idssrc=None, increasing=None, legendgroup=None, line=None, low=None, lowsrc=None, meta=None, metasrc=None, name=None, opacity=None, open=None, opensrc=None, selectedpoints=None, showlegend=None, stream=None, text=None, textsrc=None, tickwidth=None, uid=None, uirevision=None, visible=None, x=None, xaxis=None, xcalendar=None, xsrc=None, yaxis=None, **kwargs)
 |  
 |  Base class for the all trace types.
 |  
 |  Specific trace type classes (Scatter, Bar, etc.) are code generated as
 |  subclasses of this class.
 |  
 |  Method resolution order:
 |      Ohlc
 |      plotly.basedatatypes.BaseTraceType
 |      plotly.basedatatypes.BaseTraceHierarchyType
 |      plotly.basedatatyp

In [16]:
live_cache.xadd('testQueue', {'msg':'val'})
live_cache.xadd('testQueue', {'msg':'val'})
live_cache.xadd('testQueue', {'msg':'val'})
live_cache.xadd('testQueue', {'msg':'val'})

'1589007541471-0'

In [24]:
live_cache.xrange('testQueue','-','+',1)

[('1589007533846-0', {'msg': 'val'})]

NameError: name 'live_cache' is not defined